In [6]:
from alpaca_trade_api.rest import REST, TimeFrame 
import pandas as pd

In [7]:
BASE_URL = "https://paper-api.alpaca.markets" 
KEY_ID = "PKOJNYZYCLT2TS80FI7Y" 
SECRET_KEY = "fJILs7exCG5QPpNgzhrtVNLVwqa0Jwqy0V92m4bO"

In [8]:
# Instantiate REST API Connection
api = REST(key_id=KEY_ID,secret_key=SECRET_KEY,base_url="https://paper-api.alpaca.markets")

# Fetch 1Minute historical bars of Bitcoin
bars = api.get_crypto_bars("BTCUSD", TimeFrame.Minute).df
print(bars)

                          exchange      open      high       low     close  \
timestamp                                                                    
2022-05-09 05:00:00+00:00     CBSE  33639.27  33661.87  33632.15  33655.31   
2022-05-09 05:00:00+00:00     ERSX  33649.40  33652.70  33644.90  33650.40   
2022-05-09 05:01:00+00:00     CBSE  33653.35  33709.11  33653.35  33701.07   
2022-05-09 05:01:00+00:00     FTXU  33678.00  33712.00  33674.00  33712.00   
2022-05-09 05:02:00+00:00     CBSE  33700.96  33724.00  33693.83  33721.96   
...                            ...       ...       ...       ...       ...   
2022-05-09 10:28:00+00:00     CBSE  33177.84  33266.77  33100.00  33266.77   
2022-05-09 10:28:00+00:00     ERSX  33141.50  33141.50  33116.00  33116.00   
2022-05-09 10:28:00+00:00     FTXU  33183.00  33218.00  33111.00  33218.00   
2022-05-09 10:29:00+00:00     CBSE  33261.91  33288.30  33201.48  33245.95   
2022-05-09 10:29:00+00:00     FTXU  33266.00  33285.00  33220.00

In [9]:
# Filter data by exchange
bars = bars[bars.exchange == 'CBSE']
print(bars)

                          exchange      open      high       low     close  \
timestamp                                                                    
2022-05-09 05:00:00+00:00     CBSE  33639.27  33661.87  33632.15  33655.31   
2022-05-09 05:01:00+00:00     CBSE  33653.35  33709.11  33653.35  33701.07   
2022-05-09 05:02:00+00:00     CBSE  33700.96  33724.00  33693.83  33721.96   
2022-05-09 05:03:00+00:00     CBSE  33718.99  33721.64  33704.80  33711.77   
2022-05-09 05:04:00+00:00     CBSE  33711.77  33711.77  33660.93  33664.75   
...                            ...       ...       ...       ...       ...   
2022-05-09 10:25:00+00:00     CBSE  33274.39  33309.05  33236.44  33260.11   
2022-05-09 10:26:00+00:00     CBSE  33259.59  33283.26  33238.51  33256.17   
2022-05-09 10:27:00+00:00     CBSE  33255.80  33255.92  33167.62  33174.47   
2022-05-09 10:28:00+00:00     CBSE  33177.84  33266.77  33100.00  33266.77   
2022-05-09 10:29:00+00:00     CBSE  33261.91  33288.30  33201.48

In [12]:
# Create a market order to buy 1 Bitcoin
order_buy = api.submit_order('BTCUSD', qty=1, side='buy')

In [11]:
# Create a market order to sell 1 Bitcoin
order_sell = api.submit_order('BTCUSD', qty=1, side='sell')

In [13]:
# Get current position on Bitcoin. Yields error if we don't hold any
api.get_position('BTCUSD')

# Get all positions and find out if we hold any Bitcoin
positions = api.list_positions()
position_qty = 0
for p in positions:
    if p.symbol == 'BTCUSD':
        position_qty = float(p.qty)
print(position_qty)

1.0


In [14]:
def get_position(symbol):
    positions = api.list_positions()
    for p in positions:
        if p.symbol == symbol:
            return float(p.qty)
    return 0

In [15]:
import random
import time

SYMBOL = 'BTCUSD'
while True:
    # GET OUR CURRENT POSITION
    position = get_position(symbol=SYMBOL)
    
    # SCIENTIFICALLY CHECK IF WE SHOULD BUY OR SELL
    gods_say_buy = random.choice([True, False])
    print(f"Holding: {position} / Gods: {gods_say_buy}")

    #CHECK IF WE SHOULD BUY
    if position == 0 and gods_say_buy == True:
        # WE BUY ONE BITCOIN
        print('The gods have spoken:')
        print(f'Symbol: {SYMBOL} / Side: BUY / Quantity: 1')
        api.submit_order(SYMBOL, qty=1, side='buy')
    #HECK IF WE SHOULD SELL
    elif position > 0 and gods_say_buy == False:
        # WE SELL ONE BITCOIN
        print('The gods have spoken:')
        print(f'Symbol: {SYMBOL} / Side: SELL / Quantity: 1')
        api.submit_order(SYMBOL, qty=1, side='sell')
    print('Lets wait for the gods to manifest again...')
    print("*"*20)
    time.sleep(10)

Holding: 1.0 / Gods: True
Lets wait for the gods to manifest again...
********************
Holding: 1.0 / Gods: False
The gods have spoken:
Symbol: BTCUSD / Side: SELL / Quantity: 1
Lets wait for the gods to manifest again...
********************


KeyboardInterrupt: 

In [ ]:
from datetime import datetime, timedelta
import math
import time

SYMBOL = 'BTCUSD'
SMA_FAST = 12
SMA_SLOW = 24
QTY_PER_TRADE = 1

# Description is given in the article
def get_pause():
    now = datetime.now()
    next_min = now.replace(second=0, microsecond=0) + timedelta(minutes=1)
    pause = math.ceil((next_min - now).seconds)
    print(f"Sleep for {pause}")
    return pause

# Same as the function in the random version
def get_position(symbol):
    positions = api.list_positions()
    for p in positions:
        if p.symbol == symbol:
            return float(p.qty)
    return 0


# Returns a series with the moving average
def get_sma(series, periods):
    return series.rolling(periods).mean()

# Checks wether we should buy (fast ma > slow ma)
def get_signal(fast, slow):
    print(f"Fast {fast[-1]}  /  Slow: {slow[-1]}")
    return fast[-1] > slow[-1]

# Get up-to-date 1 minute data from Alpaca and add the moving averages
def get_bars(symbol):
    bars = api.get_crypto_bars(symbol, TimeFrame.Minute).df
    bars = bars[bars.exchange == 'CBSE']
    bars[f'sma_fast'] = get_sma(bars.close, SMA_FAST)
    bars[f'sma_slow'] = get_sma(bars.close, SMA_SLOW)
    return bars

while True:
    # GET DATA
    bars = get_bars(symbol=SYMBOL)
    # CHECK POSITIONS
    position = get_position(symbol=SYMBOL)
    should_buy = get_signal(bars.sma_fast,bars.sma_slow)
    print(f"Position: {position} / Should Buy: {should_buy}")
    if position == 0 and should_buy == True:
        # WE BUY ONE BITCOIN
        api.submit_order(SYMBOL, qty=QTY_PER_TRADE, side='buy')
        print(f'Symbol: {SYMBOL} / Side: BUY / Quantity: {QTY_PER_TRADE}')
    elif position > 0 and should_buy == False:
        # WE SELL ONE BITCOIN
        api.submit_order(SYMBOL, qty=QTY_PER_TRADE, side='sell')
        print(f'Symbol: {SYMBOL} / Side: SELL / Quantity: {QTY_PER_TRADE}')

    time.sleep(get_pause())
    print("*"*20)
